Nous allons détailler les fonctions dans ce notebook.

### IMPORTS

In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from pathlib import Path
import os
# from webdriver_manager.chrome import ChromeDriverManager
# import chromedriver_binary
import re
import csv

In [2]:
DRIVER_PATH_chromedriver = str(Path('../chromedriver').resolve())
browser = webdriver.Chrome(executable_path= DRIVER_PATH_chromedriver)

/tmp/ipykernel_558/2348712840.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path= DRIVER_PATH_chromedriver)


In [3]:
regex_expression = "</span> #.* in Kindle Store " # utile pour regex

### URLs exemple

In [4]:
url_kindle_accueil = "https://www.amazon.com/b?node=154606011"
url_kindle_cat_art_and_photo ="https://www.amazon.com/s?bbn=154606011&rh=n%3A133140011%2Cn%3A154606011%2Cn%3A154607011&dc&qid=1648440703&rnid=154606011&ref=lp_154606011_nr_n_0"
url_kindle_cat_art_and_photo_subcat_music_p1 = "https://www.amazon.com/s?rh=n%3A156302011&fs=true&ref=lp_156302011_sar"

url_kindle_cat_art_and_photo_subcat_umusic_p2 = "https://www.amazon.com/s?i=digital-text&rh=n%3A156302011&fs=true&page=2&qid=1648440846&ref=sr_pg_2"
url_kindle_cat_art_and_photo_subcat_umusic_p3  ="https://www.amazon.com/s?i=digital-text&rh=n%3A156302011&fs=true&page=3&qid=1648440968&ref=sr_pg_3"

In [5]:
url_livre1="https://www.amazon.com/Last-Goddess-Novel-Kate%C5%99ina-Tu%C4%8Dkov%C3%A1-ebook/dp/B09BZ8L74Q/ref=pd_sbs_sccl_1_3/131-3814768-2928133?pd_rd_w=eOyFv&pf_rd_p=3676f086-9496-4fd7-8490-77cf7f43f846&pf_rd_r=XXEVJ89KR0WC7CQF9791&pd_rd_r=ff1f8e44-e48e-4dcc-8716-a64d5675ebe6&pd_rd_wg=tAIJl&pd_rd_i=B09BZ8L74Q&psc=1"
url_livre2="https://www.amazon.com/gp/product/B09K7J47Y8/ref=s9_acss_bw_cg_bsmafr_1b1_w?pf_rd_m=ATVPDKIKX0DER&pf_rd_s=merchandised-search-13&pf_rd_r=EX2QJTNDW0DCKTRX8KFT&pf_rd_t=101&pf_rd_p=5074797f-6c32-4283-a5a7-526fd1198645&pf_rd_i=14413390011"
url_livre3="https://www.amazon.com/Healthy-Smoothie-Cookbook-Breakfast-Kid-Friendly-ebook/dp/B083QK3DT9/ref=sr_1_12?keywords=smoothie+recipe+book&qid=1651700938&s=digital-text&sprefix=smoothie%2Cdigital-text%2C236&sr=1-12"
url_yumkasi = "https://www.amazon.com/Minoru-Yamasaki-Humanist-Architecture-Modernist-ebook/dp/B0798MHV9R/ref=sr_1_17?qid=1652127157&rnid=157630011&s=digital-text&sr=1-17"

# Ouverture du navigateur

avant bien lancer le programme X launch sur windows en cochant sur disable

In [27]:
DRIVER_PATH_chromedriver = str(Path('../chromedriver').resolve())
browser = webdriver.Chrome(executable_path= DRIVER_PATH_chromedriver)
browser.get(url_kindle_accueil)

/tmp/ipykernel_533/2347370238.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path= DRIVER_PATH_chromedriver)


avec le programme ci-après j arrive à lancer chrome depuis notebook !! yes

>### il faut réussir à faire différents programme:
>   - sur la page d'un livre ( en input un url) trouver le livre , l'auteur, le ranking
>   - sur une page, trouver tout les livres de la pages
>   - sur une catégorie trouver les pages de catégories

# Fonction 1 : avec url d'un livre , retourner ranking des ventes, le nom l'auteur 

In [7]:
def encrypter(url,titre,rang,prix):
    dictionnary = [{'url':url ,'titre':titre,  'rang':rang ,'prix':prix }]
    print(dictionnary)
    with open('results.csv','a') as f :                        # ouvre le fichier result.csv
        fields=['titre','rang','prix','url']                  # permet de faire un    
        writer = csv.DictWriter(f,fieldnames = fields)
        writer.writerows(dictionnary)


In [35]:
def encryper_depuis_url_page_livre(url):
    browser.get(url)
    code_de_la_page = browser.page_source
    soup = BeautifulSoup(code_de_la_page , 'lxml')
    
    # URL du livre OK
    url = url 
    # titre du livre
    titre_avec_virgule = soup.find('span'  , {'class': 'a-size-extra-large'}).string
    titre = titre_avec_virgule.replace(",", " ")
    #rang du livre OK
    if soup.find_all( {'span'  : 'a-list-item'}) is not None : 
        rang_livre = soup.find_all( {'span'  : 'a-list-item'})
        if len(str(re.compile(regex_expression).search(str(rang_livre))).split())>5 : 
            rang = str(re.compile(regex_expression).search(str(rang_livre))).split()[5].replace("#","").replace(",","") 
        else : rang = None
    else : rang = None
    #prix du livre
    if soup.find('span' , class_ = 'a-price-whole') is not None:
        prix = soup.find('span' , class_ = 'a-price-whole').text.strip(' ').strip('.')
    else:
        prix = None
    encrypter(url,titre,rang,prix)
    

In [9]:
def encryper_pour_test(url):
    browser.get(url)
    code_de_la_page = browser.page_source
    soup = BeautifulSoup(code_de_la_page , 'lxml')
    #rang_livre = soup.find_all( {'span'  : 'a-list-item'})
    # URL du livre OK
    url = url 
    # titre du livre
    titre_avec_virgule = soup.find('span'  , {'class': 'a-size-extra-large'}).string
    titre = titre_avec_virgule.replace(",", " ")
    #rang du livre OK
    rang = '1000' #str(re.compile(regex_expression).search(str(rang_livre))).split()[5].replace("#","").replace(",","") 
    #prix du livre
    if soup.find('span' , class_ = 'a-price-whole') is not None:
        prix = soup.find('span' , class_ = 'a-price-whole').text.strip(' ').strip('.')
    else:
        prix = None
    encrypter(url,titre,rang,prix)

In [36]:
#TEST
encryper_depuis_url_page_livre("https://www.amazon.com/Intertemporal-Asset-Pricing-Contributions-Economics-ebook/dp/B000QCQT2A/ref=sr_1_1153?qid=1653945430&s=digital-text&sr=1-1153")

[{'url': 'https://www.amazon.com/Intertemporal-Asset-Pricing-Contributions-Economics-ebook/dp/B000QCQT2A/ref=sr_1_1153?qid=1653945430&s=digital-text&sr=1-1153', 'titre': ' Intertemporal Asset Pricing: Evidence from Germany (Contributions to Economics) ', 'rang': None, 'prix': None}]


# Sur une page récupérer les données : de classement de vente , nom du livre catégorie

In [ ]:
books_html = soup.find_all("article", class_="product_pod")
len(books_html)

In [ ]:
url = "https://www.amazon.com/s?i=digital-text&bbn=157630011&rh=n%3A133140011%2Cn%3A154606011%2Cn%3A154607011%2Cn%3A157630011%2Cn%3A157631011&dc&fs=true&qid=1651987607&rnid=157630011&ref=sr_nr_n_1"
url_page_1 = "https://www.amazon.com/s?i=digital-text&bbn=157630011&rh=n%3A133140011%2Cn%3A154606011%2Cn%3A154607011%2Cn%3A157630011%2Cn%3A157631011&dc&fs=true&page=1&qid=1651987613&rnid=157630011&ref=sr_pg_2"
url_page_2 = "https://www.amazon.com/s?i=digital-text&bbn=157630011&rh=n%3A133140011%2Cn%3A154606011%2Cn%3A154607011%2Cn%3A157630011%2Cn%3A157631011&dc&fs=true&page=2&qid=1651987613&rnid=157630011&ref=sr_pg_2"
url_page_3     = "https://www.amazon.com/s?i=digital-text&bbn=157630011&rh=n%3A133140011%2Cn%3A154606011%2Cn%3A154607011%2Cn%3A157630011%2Cn%3A157631011&dc&fs=true&page=3&qid=1651987613&rnid=157630011&ref=sr_pg_3"
url_page_3bis  = "https://www.amazon.com/s?i=digital-text&bbn=157630011&rh=n%3A133140011%2Cn%3A154606011%2Cn%3A154607011%2Cn%3A157630011%2Cn%3A157631011&dc&fs=true&page=3&qid=1651987793&rnid=157630011&ref=sr_pg_3"


In [ ]:
listes_des_livres=[]
browser.get(url_page_2) #navigue sur une page avec tous les lien du site
html_page_listing = browser.page_source # permet de prendre le code html de la page 
soup = BeautifulSoup(html_page_listing , 'lxml') # créer la soupe avec le parser
livres = soup.find_all('div' ,{ 'data-asin':True, 'data-component-type' : 's-search-result'} )
#tous les livres de la pages
for livre in livres:
     url_du_livre = 'https://www.amazon.com'+livre.h2.a.get('href')
     encryper_depuis_url_page_livre(url_du_livre)
    # le numéro ASIN correspeond à la ref du livre sur amazon
# ASIN=B098652LH8P

In [11]:
url_pb = "https://www.amazon.com/Quer%C3%ADamos-Calatrava-arquitect%C3%B3nicos-seducci%C3%B3n-Cr%C3%B3nicas-ebook/dp/B01LZXUQNV/ref=sr_1_20?qid=1652824900&rnid=157630011&s=digital-text&sr=1-20"

In [12]:
encryper_depuis_url_page_livre(url_pb)

[{'url': 'https://www.amazon.com/Quer%C3%ADamos-Calatrava-arquitect%C3%B3nicos-seducci%C3%B3n-Cr%C3%B3nicas-ebook/dp/B01LZXUQNV/ref=sr_1_20?qid=1652824900&rnid=157630011&s=digital-text&sr=1-20', 'titre': ' Queríamos un Calatrava. Viajes arquitectónicos por la seducción y el repudio (Crónicas nº 112) (Spanish Edition) ', 'rang': '1733491', 'prix': None}]


In [13]:
url_du_livre = 'https://www.amazon.com'+livres[2].h2.a.get('href')

NameError: name 'livres' is not defined

In [14]:
url_du_livre

NameError: name 'url_du_livre' is not defined

In [15]:
browser.get(url_du_livre)

NameError: name 'url_du_livre' is not defined

In [16]:
trouver_rang("https://www.amazon.com/Minoru-Yamasaki-Humanist-Architecture-Modernist-ebook/dp/B0798MHV9R/ref=sr_1_17?qid=1652130424&rnid=157630011&s=digital-text&sr=1-17")

NameError: name 'trouver_rang' is not defined

# Fonction permettant d'incrémenter une page web
    

In [11]:
#def lire_page_suivante(url):
url_pour_suivre ="https://www.amazon.com/s?i=digital-text&rh=n%3A157631011&fs=true&page=2&qid=1652385543&ref=sr_pg_2"


1 - réussir à couper la chaine de caractère à gauche de "&page="

In [17]:
i = 1

In [18]:
url_pour_suivre.split("&page=")[0] + "&page=" + str(i)

NameError: name 'url_pour_suivre' is not defined

In [19]:
url_a_incrementer = "https://www.amazon.com/s?i=digital-text&rh=n%3A157631011&fs=true&page=2&qid=1653250959&ref=sr_pg_2"


In [20]:
url_a_incrementer.split("&page=")[1].split("&qid=")[0]

'2'

In [21]:
def incrementer_url(url):
    url_gauche =  url.split("&page=")[0]+"&page="  # trouve la première partie de la chaine à gauche
    numero_de_la_page = str(int((url.split("&page=")[1].split("&qid=")[0]))+1) #trouve la partie numérique, convertie en nombre et ajoute 1
    return(str(url_gauche + numero_de_la_page) )

In [22]:
incrementer_url("https://www.amazon.com/s?i=digital-text&rh=n%3A157631011&fs=true&page=5")

'https://www.amazon.com/s?i=digital-text&rh=n%3A157631011&fs=true&page=6'

In [23]:
url_test = "https://www.amazon.com/s?i=digital-text&rh=n%3A157631011&fs=true&page=3"
i=0
while i < 10 :
    url_test = incrementer_url(url_test)
    print(url_test)
    i = i+1

https://www.amazon.com/s?i=digital-text&rh=n%3A157631011&fs=true&page=4
https://www.amazon.com/s?i=digital-text&rh=n%3A157631011&fs=true&page=5
https://www.amazon.com/s?i=digital-text&rh=n%3A157631011&fs=true&page=6
https://www.amazon.com/s?i=digital-text&rh=n%3A157631011&fs=true&page=7
https://www.amazon.com/s?i=digital-text&rh=n%3A157631011&fs=true&page=8
https://www.amazon.com/s?i=digital-text&rh=n%3A157631011&fs=true&page=9
https://www.amazon.com/s?i=digital-text&rh=n%3A157631011&fs=true&page=10
https://www.amazon.com/s?i=digital-text&rh=n%3A157631011&fs=true&page=11
https://www.amazon.com/s?i=digital-text&rh=n%3A157631011&fs=true&page=12
https://www.amazon.com/s?i=digital-text&rh=n%3A157631011&fs=true&page=13


In [24]:
listes_des_livres=[]
#tous les livres de la pages
for i in range(10):
    browser.get( url_pour_suivre.split("&page=")[0] + "&page=" + str(i)) #navigue sur une page avec tous les liens du site
    html_page_listing = browser.page_source # permet de prendre le code html de la page 
    soup = BeautifulSoup(html_page_listing , 'lxml') # créer la soupe avec le parser
    livres = soup.find_all('div' ,{ 'data-asin':True, 'data-component-type' : 's-search-result'} )
    for livre in livres:
        url_du_livre = 'https://www.amazon.com'+livre.h2.a.get('href')
        encryper_depuis_url_page_livre(url_du_livre) 
   

NameError: name 'url_pour_suivre' is not defined

In [25]:
url_fin = "https://www.amazon.com/s?i=digital-text&rh=n%3A157631011&fs=true&page=74"
browser.get(url_fin)
html_code = browser.page_source
soup = BeautifulSoup(html_code , 'lxml') # créer la soupe avec le parser
livres = soup.find_all('div' ,{ 'data-asin':True, 'data-component-type' : 's-search-result'} )
if livres!=[]:
    for livre in livres:
         url_du_livre = 'https://www.amazon.com'+livre.h2.a.get('href')
         encryper_depuis_url_page_livre(url_du_livre) 


In [26]:
listes_des_livres=[]
#tous les livres de la pages
for i in range(10):
    browser.get( url_pour_suivre.split("&page=")[0] + "&page=" + str(i)) #navigue sur une page avec tous les liens du site
    html_page_listing = browser.page_source # permet de prendre le code html de la page 
    soup = BeautifulSoup(html_page_listing , 'lxml') # créer la soupe avec le parser
    livres = soup.find_all('div' ,{ 'data-asin':True, 'data-component-type' : 's-search-result'} )
    for livre in livres:
        url_du_livre = 'https://www.amazon.com'+livre.h2.a.get('href')
        encryper_depuis_url_page_livre(url_du_livre) 
   

NameError: name 'url_pour_suivre' is not defined

In [27]:
import pandas as pd

In [28]:
data = pd.read_csv("results.csv")

In [29]:
data

,Titre,rang,prix,url
0,A Place of My Own: The Architecture of Daydre...,263356,10.0,https://www.amazon.com/Place-My-Own-Architectu...
1,The $12 Million Stuffed Shark: The Curious Ec...,261992,1.0,https://www.amazon.com/Million-Stuffed-Shark-E...
2,The Gardner Heist: The True Story of the Worl...,304142,12.0,https://www.amazon.com/Gardner-Heist-Worlds-La...
3,Stoicism: How to Use Stoic Philosophy to Find...,118329,3.0,https://www.amazon.com/Stoicism-Stoic-Philosop...
4,Confessions Of an Art Addict,581361,4.0,https://www.amazon.com/Confessions-Art-Addict-...
...,...,...,...,...
198,Intertemporal Asset Pricing: Evidence from Ge...,1000,NaN,https://www.amazon.com/Intertemporal-Asset-Pri...
199,Museum Volunteers: Good Practice in the Manag...,1000,NaN,https://www.amazon.com/Museum-Volunteers-Pract...
200,\n Greenwood Biographies\n,1000,NaN,https://www.amazon.com/Greenwood-Biographies/d...
201,How to Create an Online Calculator or Online ...,1000,NaN,https://www.amazon.com/Create-Calculator-Inter...


## PROGRAMME LE PLUS ABOUTI

In [52]:
DRIVER_PATH_chromedriver = str(Path('../chromedriver').resolve())
browser = webdriver.Chrome(executable_path= DRIVER_PATH_chromedriver)

/tmp/ipykernel_533/2348712840.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path= DRIVER_PATH_chromedriver)


In [37]:
url_A="https://www.amazon.com/s?i=digital-text&rh=n%3A157631011&fs=true&page=73"
url_B = 'https://www.amazon.com/s?i=digital-text&bbn=156147011&rh=n%3A133140011%2Cn%3A154606011%2Cn%3A156116011%2Cn%3A156147011%2Cn%3A16977135011&dc&fs=true&page=13'
url_C="https://www.amazon.com/s?i=digital-text&bbn=157630011&rh=n%3A133140011%2Cn%3A154606011%2Cn%3A154607011%2Cn%3A157630011%2Cn%3A157639011&dc&fs=true&page=64"
listing=[url_C , url_B , url_A ]
livres=['non nul']
for url_categorie in listing : 
    while True:
        browser.get( url_categorie) #navigue sur une page avec tous les liens du site
        html_page_listing = browser.page_source # permet de prendre le code html de la page 
        soup = BeautifulSoup(html_page_listing , 'lxml') # créer la soupe avec le parser
        livres = soup.find_all('div' ,{ 'data-asin':True, 'data-component-type' : 's-search-result'} ) # trouve tous les livres de la pages
        if livres!=[]:
            for livre in livres:
                url_du_livre = 'https://www.amazon.com'+livre.h2.a.get('href')
                encryper_depuis_url_page_livre(url_du_livre) 
        else: break
        url_categorie = incrementer_url(url_categorie)

[{'url': 'https://www.amazon.com/Holiday-Illustrated-Diaries-Llewelyn-Pritchard-ebook/dp/B009S7B5A0/ref=sr_1_1009?qid=1653972357&s=digital-text&sr=1-1009', 'titre': ' Venice Italy Holiday (The Illustrated Diaries of Llewelyn Pritchard MA Book 5) (Basque Edition) ', 'rang': None, 'prix': None}]
[{'url': 'https://www.amazon.com/Valtellina-UNESCO-Making-Global-Landscape-ebook/dp/B009GIPRNC/ref=sr_1_1010?qid=1653972357&s=digital-text&sr=1-1010', 'titre': ' The Valtellina and UNESCO: Making a Global Landscape ', 'rang': None, 'prix': None}]
[{'url': 'https://www.amazon.com/Vicchio-Mugello-Atlante-Storico-italiane-ebook/dp/B009ENY0LO/ref=sr_1_1011?qid=1653972357&s=digital-text&sr=1-1011', 'titre': ' Vicchio del Mugello (Atlante Storico delle Città italiane Vol. 12) (Italian Edition) ', 'rang': '4606021', 'prix': None}]
[{'url': 'https://www.amazon.com/Structural-Building-Conservation-Dimitris-Theodossopoulos-ebook/dp/B008SA2E9O/ref=sr_1_1012?qid=1653972357&s=digital-text&sr=1-1012', 'titre':

[{'url': 'https://www.amazon.com/Inkscapes-Pennsylvania-Timothy-George-Architect-ebook/dp/B008LZRPRW/ref=sr_1_1039?qid=1653972400&s=digital-text&sr=1-1039', 'titre': ' Hare Inkscapes Pennsylvania ', 'rang': None, 'prix': None}]
[{'url': 'https://www.amazon.com/Create-Calculator-Interactive-Worksheets-Minutes-ebook/dp/B009CT85GQ/ref=sr_1_193?qid=1653972445&s=digital-text&sr=1-193', 'titre': ' How to Create an Online Calculator or Online Interactive Worksheets in 14 Minutes ', 'rang': '3610080', 'prix': None}]
[{'url': 'https://www.amazon.com/Intertemporal-Asset-Pricing-Contributions-Economics-ebook/dp/B000QCQT2A/ref=sr_1_1153?qid=1653972452&s=digital-text&sr=1-1153', 'titre': ' Intertemporal Asset Pricing: Evidence from Germany (Contributions to Economics) ', 'rang': None, 'prix': None}]
[{'url': 'https://www.amazon.com/Museum-Volunteers-Practice-Management-Care-Preservation-Management-ebook/dp/B000OT8DZ6/ref=sr_1_1154?qid=1653972452&s=digital-text&sr=1-1154', 'titre': ' Museum Voluntee